# Phenome-Wide analysis on COPDgene data: R PIC-SURE API use-case

This notebook is an illustration example of how to use the R **PIC-SURE API** to select and query data from an HPDS-hosted database. It takes as use-case a simple PheWAS analysis. This notebook is intentionally straightforward, without too much explanation. For a more step-by-step introduction to the R PIC-SURE API, see the `R_PICSURE-API_101_PheWAS_example.ipynb` Notebook.

**Before running this notebook, please be sure to get an user-specific security token. For more information on how to proceed, see the `HPDS_connection.ipynb` notebook**

# Environment set-up

### Environment pre-requisite
- R 3.5 or later

In [ ]:
list_packages <- c("jsonlite", 
                   "ggplot2",
                   "plyr",
                   "dplyr",
                   "tidyr",
                   "purrr",
                   "stringr",
                   "ggrepel",
                   "devtools")

for (package in list_packages){
     if(! package %in% installed.packages()){
         install.packages(package, dependencies = TRUE, 
                         character.only = TRUE)
     }
     library(package, character.only = TRUE)
}

Installing R PIC-SURE API packages

In [ ]:
devtools::install_github("hms-dbmi/pic-sure-r-client")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds")

#### Loading user-defined functions

In [ ]:
source("R_lib/utils.R")

## Connection to COPDgenes database

In [ ]:
credentials_dic = jsonlite::read_json("dic_resources.json")

In [ ]:
credentials = credentials_dic[["copd"]]

In [ ]:
token <- TokenManager(credentials[["token_file"]])

In [ ]:
myconnection <- picsure::connect(url = credentials[["url"]],
                                 token = token)

In [ ]:
resource <- hpds::get.resource(myconnection,
                               resourceUUID = credentials[["resource"]])

## PheWAS analysis

In a nutshell, this PheWAS analysis consists of two main steps:
- Running univariate tests again every phenotypes variable
- Adjusting for multiple testing issue

In this example, we will select every phenotype variables available in the Dictionary, except for the variables pertaining to the "Sub-study ESP LungGO COPDGene" category (very small and specific population as compared to the COPDGene one).

### 1. Retrieving variable dictionary from HPDS Database

In [ ]:
all_variables <- hpds::find.in.dictionary(resource)

In [ ]:
variablesDict <- hpds::extract.dataframe(all_variables)
variablesDict <- variablesDict[order(variablesDict["name"]),]

In [ ]:
head(variablesDict[["name"]])

In [ ]:
## Parse variables names
library("stringr")
library("dplyr")

get_multiIndex <- function(variablesDict) {
    splitted <- gsub("^\\\\", "", variablesDict[["name"]]) %>% 
        strsplit("\\\\") 
    multiIndex <- lapply(splitted, function(x) {
        names(x) <- paste("level", 1:length(x))
        return(x)
    }) %>% do.call(dplyr::bind_rows, .)
    multiIndex[["name"]] <- variablesDict[["name"]]
    multiIndex[["simplified_name"]] <- sapply(splitted, function(x) x[length(x)])
    return(multiIndex)
}

In [ ]:
multiIndex <- get_multiIndex(variablesDict)
head(multiIndex)

In [ ]:
head(variablesDict)

### 2. Selecting variables and retrieving data from HPDS

In [ ]:
mask_pheno = variablesDict["HpdsDataType"] == "phenotypes"
mask_substudy = multiIndex[1] != "Sub-study ESP LungGO COPDGene"
mask_vars = mask_pheno & mask_substudy
selected_vars = variablesDict[mask_vars, "name"]

In [ ]:
my_query = hpds::new.query(resource = resource)
hpds::query.select.add(query = my_query, 
                      keys = selected_vars)
facts = hpds::query.run(query = my_query, result.type = "dataframe")

In [ ]:
variablesDict["name"]

### Data-management

##### Since variable names are not the same between the variable dictionary and the dataframe columns, a temporary workaround is needed: parsing variables names from variablesDict to match those retrieved from the database

In [ ]:
variablesDict[["df_name"]] <- parsing_varNames(variablesDict[["name"]])
multiIndex[["df_name"]] <- variablesDict[["df_name"]]

In [ ]:
checking_parsing(names(facts)[-1], variablesDict[mask_vars, "df_name"])

#### Selecting variables regarding their types

One important step in a PheWAS is to get the distinction between categorical and numerical variables. This distinction is straightforward using the variables dictionary.

In [ ]:
mask_categories <- variablesDict[, "categorical"] == TRUE
categorical_varnames <- variablesDict[mask_categories & mask_vars, "df_name"]
continuous_varnames <- variablesDict[!mask_categories & mask_vars, "df_name"]

### Selecting the dependent variable to study
Most of PheWAS use a genetic variant as the variable used to separate the population between cases and controls. But the population doesn't have to be dichotomized using a genetic variant, and any phenotypic variable could be used (see for example [*Neuraz et al.*, 2013](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003405)). 

Here we will use the presence or absence of a COPD diagnosis as the variable to dichotomize the population in our subsequent analysis.

In [ ]:
dependent_var_name <- multiIndex[multiIndex[["simplified_name"]] == "00 Affection status",][["df_name"]]
categorical_varnames <- categorical_varnames[-which(categorical_varnames == dependent_var_name)]

In [ ]:
table(facts[[dependent_var_name]])

Then we subset our population regarding the relevant values for the COPD diagnosis variable (i.e. keeping "Case" and "Control" individuals, thus discarding "Other", "Control, Exclusionary Disease", and null values).

In [ ]:
mask_dependent_var_name = facts[[dependent_var_name]] %in% c("Case", "Control")

In [ ]:
facts <- facts[mask_dependent_var_name,]

In [ ]:
count_case_control <- table(facts[[dependent_var_name]])

In [ ]:
sprintf("Control: %i individuals\nCase: %i individuals",
        count_case_control["Control"],
        count_case_control["Case"]) %>% cat()

### Univariate statistical tests

In [ ]:
## Unified univariate tests
### Significance of association tests are retrieved from the p-values of a Likelihood ratio test
anova_model <- function(data, dependent_var, independent_var) {
    model <- glm(as.formula(paste(dependent_var, "~ 1 +", independent_var)),
                 data = data,
                 family = binomial(link="logit"))
    model_reduced <- glm(as.formula(paste(dependent_var, "~ 1")),
                         data = data,
                         family = binomial(link="logit"))
    p_val <- anova(model, model_reduced, test =  "LRT")[2, "Pr(>Chi)"]
    return(p_val)    
}

In [ ]:
independent_var_names = c(categorical_varnames, continuous_varnames)

In [ ]:
pvalues_list = list()
error_list =  list()
warning_list = list()
## Errors storing is not working yet, get back to it later
for (independent_var_name in independent_var_names) {
    data <- na.omit(facts[,c(dependent_var_name, independent_var_name)])
    tryCatch({
                pvalues_list[[independent_var_name]] <- anova_model(data, dependent_var_name, independent_var_name)
#                error_list[[independent_var_name]] <- NA
 #               warning_list[[independent_var_name]] <- NA
    },
             error = function(e) {
                print(paste("error", independent_var_name))
                pvalues_list[[independent_var_name]] <- NA
                error_list[[independent_var_name]] <- e
                warning_list[[independent_var_name]] <- NA
             },
             warning = function(w) {
                print(paste("warning", independent_var_name))

                pvalues_list[[independent_var_name]] <- NA
                error_list[[independent_var_name]] <- NA
                warning_list[[independent_var_name]] <- w                 
             }
        )
}

In [ ]:
df_pvalues <- data.frame(
    "df_name" = names(pvalues_list),
    "pvalues" = simplify(unname(pvalues_list)), 
    stringsAsFactors = F
)

In [ ]:
df_pvalues[["log_pvalues"]] <- -log10(df_pvalues$pvalues)

In [ ]:
multiIndex_enhanced <- dplyr::left_join(multiIndex, df_pvalues, by="df_name")
variablesDict_enhanced <- dplyr::left_join(variablesDict, df_pvalues, by="df_name")

In [ ]:
variablesDict_enhanced

### Results visualization

#### Univariate tests distribution

In [ ]:
ggplot(aes_string(x = "pvalues", 
                 fill = "categorical"),
       data = variablesDict_enhanced) +
geom_histogram(bins=20, position = "dodge") +
scale_fill_brewer(palette='Paired') + 
labs(title = "Distribution of non-adjusted p-values among tested phenotypes ", 
    subtitle = expression(italic("Likelihood Ratio Test"))) +
xlab("Unadjusted p-values") +
ylab("Count") +
theme_bw()

#### Multiple hypotheses testing correction: Bonferroni Method

In order to handle the multiple comparison issue (increase in the probability to "discover" false statistical associations, because of the number of tests performed), we will use the Bonferroni correction method. Although many other multiple comparison exist, Bonferroni is the most straightforward to use, because it doesn't require assumptions about variables correlation. Other PheWAS analysis also use False Discovery Rate controlling procedures ([see](https://en.wikipedia.org/wiki/False_discovery_rate)).

In a nutshell, Bonferonni allows to calculate a corrected "statistical significant threshold" according to the number of test performed. Every p-value below this threshold will be deemed statistically significant.

In [ ]:
variablesDict_enhanced$adj_pvalues <- p.adjust(variablesDict_enhanced$pvalues, method="bonferroni")

In [ ]:
variablesDict_enhanced$log_adj_pvalues <- -log10(variablesDict_enhanced$adj_pvalues)

## Manhattan plot

#### Preparing data

In [ ]:
corrected_alpha <- 0.05/length(variablesDict_enhanced$pvalues) # Using Bonferonni method
adj_corrected_alpha <- -log10(corrected_alpha)

In [ ]:
non_nan <- which(!is.na(variablesDict_enhanced$pvalues))
plot_df <- multiIndex_enhanced[non_nan, ]
plot_df$df_name <- as.factor(plot_df$df_name)
plot_df$log_pvalues <- round(plot_df$log_pvalues, 5)

plot_df = multiIndex[, c("df_name", "level 2")] %>% 
plyr::rename(replace = c("level 2" = "category")) %>%
right_join(plot_df, by="df_name")

plot_df <- plot_df[order(plot_df$category),]
plot_df$category <- factor(plot_df$category)
plot_df$name <- factor(plot_df$name, levels=plot_df$name[order(plot_df$category)])

In [ ]:
get_breaks <- function(plot_df) {
    number_rows <- nrow(plot_df)
    plot_df$row_num <- 1:number_rows
    plot_df$center <- NA
    for (cat in unique(plot_df$category)) {
        mask <- plot_df$category == cat
        mask[is.na(mask)] <- F
        center = mean(plot_df[mask, ][["row_num"]]) %>% round(0)

        plot_df[mask, "center"] <- center
    }
    return(plot_df)
}
plot_df <- get_breaks(plot_df)

In [ ]:
# Selecting 4 largest p-values, to be annotated in the Manatthan plot

largest_pvalues_indices <- order(plot_df[["log_pvalues"]], decreasing=T)[1:4]
plot_df$to_annotate <- "no"
plot_df[largest_pvalues_indices, "to_annotate"] <- "yes"

In [ ]:
# Suppressing Inf log(p-values)
plot_df <- plot_df[plot_df$log_pvalues != Inf,]

#### Plotting the data

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

In [ ]:
my_theme <- theme_bw() + 
theme(axis.title.y = element_text(face="italic", size=15),
      title = element_text(size=20),
      axis.title.x = element_text(size=15),
      axis.text.x = element_text(angle=35, hjust=1),
      legend.position = "none",
      panel.grid.major.x = element_blank()
      )


In [ ]:
paired_colors <- c("navyblue", "lightskyblue")

In [ ]:
# Manatthan plot using geom_jitter --> more suitable when number of point per category is enough
ggplot(plot_df, aes(x=category, y=log_pvalues)) +
geom_jitter(alpha=1, aes_string(colour="category"), 
           width=0.5,
           size=3) +
geom_hline(yintercept=adj_corrected_alpha, linetype="dashed") +
scale_y_continuous(expand = c(0, 0) ) +
scale_color_manual(values = rep(paired_colors, times=20)) +
geom_label_repel( data=subset(plot_df, to_annotate=="yes"), aes(label=simplified_name), size=3.5) +
labs(title="Association between phenotypes variables and gene mutation (COPD status)", 
    x="Phenotypes", 
    y="- log10(p-values)",
    colour="Phenotypes categories") +
my_theme



In [ ]:
# Manatthan plot using geom_point --> workaround when not that much points
manatthan_theme <- theme_bw() + 
theme(axis.title.y = element_text(face="italic", size=15),
      title = element_text(size=20),
      axis.title.x = element_text(size=15),
      axis.text.x = element_text(angle=60, hjust=1, size= 10),
      legend.position = "none",
      panel.grid.major.x = element_blank()
      )

ggplot(plot_df, aes(x=row_num, y=log_pvalues)) +
geom_point(alpha=1,
           aes_string(colour="category"),
           size=3) +
geom_hline(yintercept=adj_corrected_alpha, linetype="dashed") +
scale_y_continuous(expand = c(0, 0) ) +
scale_color_manual(values = rep(paired_colors, times=20)) +
geom_label_repel( data=subset(plot_df, to_annotate=="yes"), aes(label=simplified_name), size=3.5) +
labs(title="Association between phenotypes variables and gene mutation (COPD status)", 
    x="Phenotypes", 
    y="- log10(p-values)",
    colour="Phenotypes categories") +
manatthan_theme +
scale_x_discrete(limits=unique(plot_df$center), labels=unique(plot_df$category))

Overall, it appears that most of the tested phenotypes covariates are above the adjusted threshold of significant association. However, it is not surprising at all, given the nature of our dependent variable: a lot of those variables are by nature tied directly to the COPD status. For instance, the 4 highest p-values (distance walked in feet, nebulizer for inhaled medication, too breathless to leave the house) are direct consequences of COPD disease.

This code can be used directly with any other variable present in the variable Dictionary. It only need to change the `dependent_var_name` value.